In [40]:
from bs4 import BeautifulSoup
import pandas as pd
import networkx as nx
import re
import os
import matplotlib.pyplot as plt
from networkx.drawing.nx_pydot import write_dot
from tqdm.notebook import tqdm

In [41]:
RCDATA = "data/cos_course_data_raw.xml"

In [42]:
with open(RCDATA) as cdata:
    s = "".join(cdata.readlines())
    tree = BeautifulSoup(s)

In [43]:
rows = []
for dep in tree.findAll("courses"):
    for course in dep.findAll("course"):
        rows.append({
            "code": getattr(course.find("code"), "string", None),
            "name": getattr(course.find("name"), "string", None),
            "credits": getattr(course.find("credits"), "string", None),
            "credit-structure": getattr(course.find("credit-structure"), "string", None),
            "pre-requisites": getattr(course.find("pre-requisites"), "string", None),
            "overlap": getattr(course.find("overlap"), "string", None),
            "department": dep.get("department"),
            "description": getattr(course.find("description"), "string", None)
        })
        
df = pd.DataFrame(rows)

In [44]:
df

,code,name,credits,credit-structure,pre-requisites,overlap,department,description
0,APL100,Engineering Mechanics,4,3-1-0,None,None,Department of Applied Mechanics,"Kinematics, Statics, Equations of Motion, Rigi..."
1,APL101,Applied Mathematics in Engineering Applications,3,3-0-0,None,None,Department of Applied Mechanics,ordinary Differential Equation: Second order o...
2,APL102,Introduction to Materials Science and Engineering,4,3-0-2,None,None,Department of Applied Mechanics,Structure of Solids: atomic and inter-atomic b...
3,APL103,Experimental Methods,4,3-0-2,None,None,Department of Applied Mechanics,Experimental Analysis: Types of measurements a...
4,APL104,Solid Mechanics,4,3-1-0,APL100,"APL105, APL108",Department of Applied Mechanics,"Introduction, State of stress at a point, equa..."
...,...,...,...,...,...,...,...,...
2387,JOL794,Selected Topics-II,3,3-0-0,None,None,Department of Textile and Fibre Engineering,None
2388,JOS795,Independent Study,3,0-3-0,None,None,Department of Textile and Fibre Engineering,None
2389,JOV796,Selected Topics in Photonics,1,1-0-0,None,None,Department of Textile and Fibre Engineering,None
2390,JOD801,Major Project Part-I,6,0-0-12,None,None,Department of Textile and Fibre Engineering,None


## Pre-Processing
Add year + semwise data

In [38]:
YEARS = ["2122", "2021", "1920"]
COURSES_OFFERED_PATH = "data/courses_offered_anon"
year = YEARS[2]
i = 2
for year in YEARS:
    for i in [1,2]:
        sem_yr_data = pd.read_csv(f"{COURSES_OFFERED_PATH}/{i}_{year}.csv")
        sem_yr_data = sem_yr_data.rename(columns={'Course Code': 'code'})
        # drop all courses with the slot as SU1 or X
        sem_yr_data = sem_yr_data.loc[(sem_yr_data['Slot Name'] != 'X') & (~sem_yr_data['Slot Name'].str.startswith('SU'))]
        df = df.merge(sem_yr_data[['code','Slot Name', 'Vacancy', 'Current Strength']],on='code',how='left')
        df = df.rename(columns={'Slot Name': f"{i}_{year}_slot", 'Vacancy': f"{i}_{year}_vacancy", 'Current Strength': f"{i}_{year}_strength"})

df

,code,name,credits,credit-structure,pre-requisites,overlap,department,description,1_2122_slot,1_2122_vacancy,...,1_2021_strength,2_2021_slot,2_2021_vacancy,2_2021_strength,1_1920_slot,1_1920_vacancy,1_1920_strength,2_1920_slot,2_1920_vacancy,2_1920_strength
0,APL100,Engineering Mechanics,4,3-1-0,None,None,Department of Applied Mechanics,"Kinematics, Statics, Equations of Motion, Rigi...",A,500.0,...,24.0,A,600.0,0.0,A,500.0,589.0,A,250.0,568.0
1,APL101,Applied Mathematics in Engineering Applications,3,3-0-0,None,None,Department of Applied Mechanics,ordinary Differential Equation: Second order o...,B,200.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,APL102,Introduction to Materials Science and Engineering,4,3-0-2,None,None,Department of Applied Mechanics,Structure of Solids: atomic and inter-atomic b...,NaN,NaN,...,371.0,E,225.0,149.0,E,350.0,307.0,E,300.0,134.0
3,APL103,Experimental Methods,4,3-0-2,None,None,Department of Applied Mechanics,Experimental Analysis: Types of measurements a...,F,150.0,...,NaN,NaN,NaN,NaN,F,150.0,97.0,NaN,NaN,NaN
4,APL104,Solid Mechanics,4,3-1-0,APL100,"APL105, APL108",Department of Applied Mechanics,"Introduction, State of stress at a point, equa...",D,300.0,...,178.0,NaN,NaN,NaN,D,150.0,153.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2747,JOL794,Selected Topics-II,3,3-0-0,None,None,Department of Textile and Fibre Engineering,None,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2748,JOS795,Independent Study,3,0-3-0,None,None,Department of Textile and Fibre Engineering,None,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2749,JOV796,Selected Topics in Photonics,1,1-0-0,None,None,Department of Textile and Fibre Engineering,None,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2750,JOD801,Major Project Part-I,6,0-0-12,None,None,Department of Textile and Fibre Engineering,None,NaN,NaN,...,13.0,P,50.0,0.0,P,60.0,23.0,NaN,NaN,NaN


In [45]:
pd.concat(g for _, g in df.groupby("code") if len(g) > 1)

,code,name,credits,credit-structure,pre-requisites,overlap,department,description
832,ELL751,Appliance Systems,3,3-0-0,"ELL203, ELL332 ELL365",None,Department of Electrical Engineering,"Introduction to Domestic Appliances, Embedded ..."
833,ELL751,Power Electronic Converters,3,3-0-0,None,None,Department of Electrical Engineering,Introduction to various power switching device...
1004,HUP102,Psychology Laboratory,1,0-0-2,None,None,Department of Humanities and Social Sciences,To familiarize students with psychological con...
1233,HUP102,Research Participation,1,0-0-1,None,None,Department of Humanities and Social Sciences,This course will expose students to various ex...
1910,JOD801,Major Project Part-I,6,0-0-12,None,None,Department of Physics,Analysis/Design/Simulation/Experimental study ...
2390,JOD801,Major Project Part-I,6,0-0-12,None,None,Department of Textile and Fibre Engineering,None
1911,JOD802,Major Project Part-II,12,0-0-24,None,None,Department of Physics,Detailed investigations on the study of contem...
2391,JOD802,Major Project Part-II,12,0-0-24,None,None,Department of Textile and Fibre Engineering,0-0-24
1903,JOL793,Selected Topics-I,3,3-0-0,None,None,Department of Physics,None
2386,JOL793,Selected Topics-I,3,3-0-0,None,None,Department of Textile and Fibre Engineering,None


Naive bayes prediction: Say a course has historically been offered in two slots, A and B. The probability that the course is offered in slot A, given that it's offered is
$$P\left(A|\text{Off}\right) = \frac{P(A)\cdot P(\text{Off}|A)}{P(Off)} = \frac{P(A)}{P(\text{Off})} = \frac{n(A)}{n(\text{Off})}$$

This is easy enough to compute and predict for all courses.

We'll need to do something different for years, though. First year cores alternate, while programme cores (CVL/SBL) are offered in every sem. Department cores generally alternate. DE's/PE's are the main targets of this. We can do a similar classification to the above for them.

In [ ]:
#
# calculate the offering scores for the different courses.
# it's ok to do the list comprehension: not too bad a performance hit here.
#

for y in years:
    for i in [1,2]:
        sltdata = df[f'{i}_{y}_slot']